### Scraping from tamil lyrics from web

#### Import

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:

emptyDict = {"பாடல்" : [], "படம்" : [], "வருடம்" : [], "இசை" : [], "பாடலாசிரியர்" : [], "பாடகர்" : [], "வரி" : []}
lyricDf = pd.DataFrame(emptyDict)
lyricDf.head()

,பாடல்,படம்,வருடம்,இசை,பாடலாசிரியர்,பாடகர்,வரி


#### Connect request

In [3]:
BASE_URL = "https://www.tamilbeatslyrics.com/category/years/2022"
result = requests.get(BASE_URL)
indexDoc = BeautifulSoup(result.text, "html.parser")
print(indexDoc.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://gmpg.org/xfn/11" rel="profile"/>
  <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
   <!-- This site is optimized with the Yoast SEO plugin v19.13 - https://yoast.com/wordpress/plugins/seo/ -->
   <title>
    Latest Tamil Song Lyrics of 2022 - Tamil Beats Lyrics
   </title>
   <link href="https://www.tamilbeatslyrics.com/category/years/2022" rel="canonical">
    <link href="https://www.tamilbeatslyrics.com/category/years/2022/page/2" rel="next"/>
    <meta content="en_US" property="og:locale">
     <meta content="article" property="og:type"/>
     <meta content="Latest Tamil Song Lyrics of 2022 - Tamil Beats Lyrics" property="og:title"/>
     <meta content="https://www.tamilbeatslyrics.com/category/years/2022" property="og:url"/>
     <meta content="Tamil Beats Lyrics"

#### Pagination get count

In [4]:
maximumPageCount = int(indexDoc.findAll("a", attrs={"class" : "page-numbers"})[1].string)
maximumPageCount

6

#### Traverse page and Fetch the song details
This while take some time

In [5]:
currentPage = 1

while(currentPage <= maximumPageCount):
    currentURL = BASE_URL + "/page/" + str(currentPage) ## navigate pagination URL
    cpResult = requests.get(currentURL)
    cpDoc = BeautifulSoup(cpResult.text, "html.parser")
    headings = cpDoc.findAll("a", attrs={"rel" : "bookmark"})


    ### go through the each page of songs lyrics
    for item in headings:
        songURL = (item.attrs)["href"]
        songResult = requests.get(songURL) ## navigate to the song details URL
        songDoc = BeautifulSoup(songResult.text, "html.parser") 
        songDict = []
        tds = songDoc.table.find_all("td")

        ### travel thorggh the table to get details of songs
        for idx in range(6):
            songDict.append(tds[idx].text)

        lyricsParas = songDoc.find_all("p", attrs = {"class" : "has-text-align-center"})

        ### merge lyrics in single sting
        lyrics = ""
        for lyric in lyricsParas:
            lyrics += lyric.getText(separator=" ") + " "
        songDict.append(lyrics)
        
        # Add to the main lyrics frame
        lyricDf.loc[len(lyricDf), lyricDf.columns] = songDict

    print("Page " + str(currentPage) + " completed ...")
    currentPage += 1

lyricDf.shape


Page 1 completed ...
Page 2 completed ...
Page 3 completed ...
Page 4 completed ...
Page 5 completed ...
Page 6 completed ...


(87, 7)

#### Export corpus as CSV

In [6]:
lyricDf.to_csv('Tamil New Songs Lyrics.csv', index=False,  encoding='utf-8')


#### Export corpus as JSON

In [7]:
with open('Tamil New Songs Lyrics.json', 'w', encoding='utf-8') as fileJSON:
    lyricDf.to_json(fileJSON, force_ascii=False)